In [1]:
from confluent_kafka import Producer
import socket
#Initialize Your Parameters here - Keep the variable values as is for the ones you can't find on the Confluent-Kafka connection 
KAFKA_CONFIG = {
    "bootstrap.servers":"pkc-6ojv2.us-west4.gcp.confluent.cloud:9092",
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"ZIGFIPPZDQBDNKRN",
    "sasl.password":"f8gyTRboB8kvt6OXO8GmjGI4sUdbC72C2avIJCr9FYsZYBmKagc+ljNQoQJnLs9m",
    "session.timeout.ms":"45000",
    "group.id":"python-group-1",
    'auto.offset.reset': 'smallest',
    'client.id': socket.gethostname()
}


topic_name = "pyspark_topic"
producer = Producer(KAFKA_CONFIG)


In [2]:
import feedparser
import time

# We are searching for tcp+udp+icmp
feed_url = "https://news.google.com/rss/search?q=popular+cyber+attacks"

# Set the start time
start_time = time.time()

# Define the duration (in seconds) for which the listener should run (5 minutes = 300 seconds)
duration = 300

def extract_news_feed(feed_url, duration):
    feed = feedparser.parse(feed_url)
    articles = []
    extracted_articles = set()
    
    while (time.time() - start_time) < duration :
        for entry in feed.entries:
            link = entry.link
            title = entry.title.encode('ascii', 'ignore').decode()
            unique_id = f'{link}-{title}'
            if unique_id in extracted_articles:
                continue
            extracted_articles.add(unique_id)
            article_data = {"title": title, "link": link}
            if article_data is not None:
                producer.produce(topic_name, key=article_data["title"], value=article_data["link"])
        producer.flush()

extract_news_feed(feed_url, duration)
